# Shamir Secret Sharing

Cicada implements Shamir secret sharing as well. This scheme, distinct from additive secret sharing, is based on evaluating and interpolating polynomials over an integral field. Shamir's scheme is a threshold scheme such that for any set of $n$ parties, a threshold $k$ can be defined such that any subset of the $n$ parties of size at least $k$ can reconstruct secrets and carry on with at least some operations. We have defined two protocols to handle and express this. All the available operations can be done with an instance of the ShamirProtocol class, but this class will throw an error if it is instantiated with a $k$ that cannot handle all of it's operations. The ShamirBasicProtocol will be able to handle greater flexibility w.r.t. $k$ though not all the operations are available. In general polynomials of the following form over the field $\mathbb{Z}_p$ are created and evaluated to generate secrets of some value $s$ shared among $n$ parties such that any subset of them of size at least $k$ will be able to reconstruct the shared secrets: 

$$
f(x)=(c_{k-1}x^{k-1}+c_{k-2}x^{k-2}+\dots+c_{2}x^{2}+c_{1}x+s)\mod p
$$

Where all the $c$ coefficients are random field elements. Reconstruction of the secrets is done by straightforward integral Lagrange interpolation. In the following, $x$ and $y$ with subscripts have their usual meaning in the context of successive points at which a polynomial is evaluated and the result of that evaluation respectivly, and the subscripts just denote the position of each within a list of such pairs. 

$$ s= f(0)=\sum_{j=1}^{n}y_j \prod_{k=1:k\neq j}^{n} \frac{-x_k}{x_j-x_k}$$

Let's consider an example of the use of ShamirBasicProtocol:

In [5]:
import logging

import numpy

import cicada.shamir
from cicada.communicator import SocketCommunicator

logging.basicConfig(level=logging.INFO)
globe_const_vals =  numpy.array([0, -1, 2, -3, 4])
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirBasicProtocol(communicator, threshold =3)

    values = globe_const_vals if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} values: {values}")
    
    values_share = protocol.share(secret=protocol.encoder.encode(values), src=0, shape=(5,))
    revealed_share = protocol.encoder.decode(protocol.reveal(values_share))
    log.info(f"Player {communicator.rank} revealed: {revealed_share}")
    dub_vals = protocol.add(values_share, values_share)
    revealed_dubs = protocol.encoder.decode(protocol.reveal(dub_vals))
    log.info(f"Player {communicator.rank} revealed dubs: {revealed_dubs}")

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 values: [ 0 -1  2 -3  4]
INFO:root:Player 1 values: None
INFO:root:Player 2 values: None
INFO:root:Player 3 values: None
INFO:root:Player 4 values: None
INFO:root:Player 0 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 1 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 2 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 3 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 4 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 0 revealed dubs: [ 0. -2.  4. -6.  8.]
INFO:root:Player 1 revealed dubs: [ 0. -2.  4. -6.  8.]
INFO:root:Player 2 revealed dubs: [ 0. -2.  4. -6.  8.]
INFO:root:Player 3 revealed dubs: [ 0. -2.  4. -6.  8.]
INFO:root:Player 4 revealed dubs: [ 0. -2.  4. -6.  8.]


If we try to do something like multiplication, or anything dependent on multiplication, with the ShamirBasicProtocol it will fail since multiplication is not made available there. ShamirProtocol has stricter requirements on the relationship between the threshold $k$ and the number of total parties related to the requirements of multiplication. 

In [6]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirBasicProtocol(communicator, threshold =3)

    values = globe_const_vals if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} values: {values}")
    
    values_share = protocol.share(secret=protocol.encoder.encode(values), src=0, shape=(5,))
    revealed_share = protocol.encoder.decode(protocol.reveal(values_share))
    log.info(f"Player {communicator.rank} revealed: {revealed_share}")
    sq_vals = protocol.untruncated_multiply(values_share, values_share)
    truncd_sq = protocol.truncate(sq_vals)
    revealed_sq= protocol.encoder.decode(protocol.reveal(truncd_sq))
    
    log.info(f"Player {communicator.rank} revealed dubs: {revealed_sq}")

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 values: [ 0 -1  2 -3  4]
INFO:root:Player 1 values: None
INFO:root:Player 2 values: None
INFO:root:Player 3 values: None
INFO:root:Player 4 values: None
INFO:root:Player 0 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 1 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 2 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 3 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 4 revealed: [ 0. -1.  2. -3.  4.]
ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 0 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/home/kgoss/git/cicada-mpc/cicada/communicator/socket/__init__.py", line 796, in launch
    result = fn(communicator, *args, **kwargs)
  File "<ipython-input-6-d642103d3f99>", line 11, in main
    sq_vals = protocol.untruncated_multiply(values_share, values_share)
AttributeError: 'ShamirBasicProtocol' object has no at

We'll try again with the SmairProtocol to see that it succeeds:

In [7]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold =3)

    values = globe_const_vals if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} values: {values}")
    
    values_share = protocol.share(secret=protocol.encoder.encode(values), src=0, shape=(5,))
    revealed_share = protocol.encoder.decode(protocol.reveal(values_share))
    log.info(f"Player {communicator.rank} revealed: {revealed_share}")
    sq_vals = protocol.untruncated_multiply(values_share, values_share)
    truncd_sq = protocol.truncate(sq_vals)
    revealed_sq= protocol.encoder.decode(protocol.reveal(truncd_sq))
    
    log.info(f"Player {communicator.rank} revealed dubs: {revealed_sq}")

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 values: [ 0 -1  2 -3  4]
INFO:root:Player 1 values: None
INFO:root:Player 2 values: None
INFO:root:Player 3 values: None
INFO:root:Player 4 values: None
INFO:root:Player 0 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 1 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 2 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 3 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 4 revealed: [ 0. -1.  2. -3.  4.]
INFO:root:Player 0 revealed dubs: [ 0.  1.  4.  9. 16.]
INFO:root:Player 1 revealed dubs: [ 0.  1.  4.  9. 16.]
INFO:root:Player 2 revealed dubs: [ 0.  1.  4.  9. 16.]
INFO:root:Player 3 revealed dubs: [ 0.  1.  4.  9. 16.]
INFO:root:Player 4 revealed dubs: [ 0.  1.  4.  9. 16.]


Note however that if you try to instantiate a ShamirProtocol with too large a threshold $k$ for the world_size (or total number of players $n$) it will throw and exception:

In [8]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold =5)

    values = globe_const_vals if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} values: {values}")
    
    values_share = protocol.share(secret=protocol.encoder.encode(values), src=0, shape=(5,))
    revealed_share = protocol.encoder.decode(protocol.reveal(values_share))
    log.info(f"Player {communicator.rank} revealed: {revealed_share}")
    sq_vals = protocol.untruncated_multiply(values_share, values_share)
    truncd_sq = protocol.truncate(sq_vals)
    revealed_sq= protocol.encoder.decode(protocol.reveal(truncd_sq))
    
    log.info(f"Player {communicator.rank} revealed squares: {revealed_sq}")

SocketCommunicator.run(world_size=5, fn=main);

ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 0 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/home/kgoss/git/cicada-mpc/cicada/communicator/socket/__init__.py", line 796, in launch
    result = fn(communicator, *args, **kwargs)
  File "<ipython-input-8-283316de5eb2>", line 3, in main
    protocol = cicada.shamir.ShamirProtocol(communicator, threshold =5)
  File "/home/kgoss/git/cicada-mpc/cicada/shamir.py", line 557, in __init__
    raise ValueError(f'Threshold incompatible with worldsize, multiplications will not be feasible. Multiplications with this threshold would require worldsize at least {2*self.d+1}. Increase worldsize or decrease threshold.')
ValueError: Threshold incompatible with worldsize, multiplications will not be feasible. Multiplications with this threshold would require worldsize at least 9. Increase worldsize

Shamir offers more direct and straightforward resilience against data loss due to some party failing, though it is often less resistant to large coalitions since the resilience comes at the cost of proper subsets of parties being able to reconstruct secrets. Additionally, greater care and attention to detail is required since the threshold relationship to the total number of parties directly controls which version of Shamir's sheme we have implmeneted is usable due to the need for a greater number of parties above the threshold if multiplications or derivitive operations are desired. In the case that these additional operations are needed, the threshold $k$ and total number of parties in the scheme must satisfy the following reltionship:

$$ k \leq \left\lfloor\frac{n}{2}\right\rfloor +1$$